## Introduction
-----

In this assignment you will recursively estimate the position of a vehicle along a trajectory using available measurements and a motion model. 

The vehicle is equipped with a very simple type of LIDAR sensor, which returns range and bearing measurements corresponding to individual landmarks in the environment. The global positions of the landmarks are assumed to be known beforehand. We will also assume known data association, that is, which measurment belong to which landmark.

## Motion and Measurement Models
-----

### Motion Model

The vehicle motion model recieves linear and angular velocity odometry readings as inputs, and outputs the state (i.e., the 2D pose) of the vehicle:

\begin{align}
\mathbf{x}_{k} &= \mathbf{x}_{k-1} + T
\begin{bmatrix}
\cos\theta_{k-1} &0 \\
\sin\theta_{k-1} &0 \\
0 &1
\end{bmatrix}
\left(
\begin{bmatrix}
v_k \\
\omega_k
\end{bmatrix}
+ \mathbf{w}_k
\right)
\, , \, \, \, \, \, \mathbf{w}_k = \mathcal{N}\left(\mathbf{0}, \mathbf{Q}\right)
\end{align}

- $\mathbf{x}_k = \left[ x \, y \, \theta \right]^T$ is the current 2D pose of the vehicle
- $v_k$ and $\omega_k$ are the linear and angular velocity odometry readings, which we use as inputs to the model

The process noise $\mathbf{w}_k$ has a (zero mean) normal distribution with a constant covariance $\mathbf{Q}$.

### Measurement Model

The measurement model relates the current pose of the vehicle to the LIDAR range and bearing measurements $\mathbf{y}^l_k = \left[r \, \phi \right]^T$.

\begin{align}
\mathbf{y}^l_k =
\begin{bmatrix}
\sqrt{(x_l - x_k - d\cos\theta_{k})^2 + (y_l - y_k - d\sin\theta_{k})^2} \\
atan2\left(y_l - y_k - d\sin\theta_{k},x_l - x_k - d\cos\theta_{k}\right) - \theta_k
\end{bmatrix}
+
\mathbf{n}^l_k
\, , \, \, \, \, \, \mathbf{n}^l_k = \mathcal{N}\left(\mathbf{0}, \mathbf{R}\right)
\end{align}

- $x_l$ and $y_l$ are the ground truth coordinates of the landmark $l$
- $x_k$ and $y_k$ and $\theta_{k}$ represent the current pose of the vehicle
- $d$ is the known distance between robot center and laser rangefinder (LIDAR)

The landmark measurement noise $\mathbf{n}^l_k$ has a (zero mean) normal distribution with a constant covariance $\mathbf{R}$.

## Getting Started
-----

Since the models above are nonlinear, we recommend using the extended Kalman filter (EKF) as the state estimator.
Specifically, you will need to provide code implementing the following steps:
- the prediction step, which uses odometry measurements and the motion model to produce a state and covariance estimate at a given timestep, and
- the correction step, which uses the range and bearing measurements provided by the LIDAR to correct the pose and pose covariance estimates

### Unpack the Data
First, let's unpack the available data:

In [ ]:
import pickle
import numpy as np
from numpy.linalg import inv
import matplotlib.pyplot as plt

with open('data/data.pickle', 'rb') as f:
    data = pickle.load(f)

t = data['t']  # timestamps [s]

x_init  = data['x_init'] # initial x position [m]
y_init  = data['y_init'] # initial y position [m]
th_init = data['th_init'] # initial theta position [rad]
print("Initial x,y and theta")
print(x_init)
print(y_init)
print(th_init)

# input signal
v  = data['v']  # translational velocity input [m/s]
om = data['om']  # rotational velocity input [rad/s]
# print(data)
#print("\nInput Signals - translational velocity")
#print(v)
# print("\nInput Signals - angular velocity")
# print(om)

# bearing and range measurements, LIDAR constants
b = data['b']  # bearing to each landmarks center in the frame attached to the laser [rad]
r = data['r']  # range measurements [m]
l = data['l']  # x,y positions of landmarks [m]
d = data['d']  # distance between robot center and laser rangefinder [m]
# print("\nBearing to each Landmark\n",b)
# print("\nRange Measurements \n",r)
# print("\nx,y positions of landmarks\n",l)
# print("\nDistance between robot centre and laser sensor\n",d)

Note that distance from the LIDAR frame to the robot center is provided and loaded as an array into the `d` variable.

### Ground Truth
If available, it is useful to plot the ground truth position and orientation before starting the assignment.

<table><tr>
<td> <img src="data/gtruth.png" alt="Ground Truth" width="350"/> </td>
<td> <img src="data/gtruth2.png" alt="Ground Truth" width="350"/> </td>
</tr></table>

Notice that the orientation values are wrapped to the $\left[-\pi,\pi\right]$ range in radians.

In [ ]:
import sympy as sp
from sympy import *
x_l, y_l, x_k, y_k, d_s, x, y, x_k1, y_k1, theta_k, theta_k1,T, v_k, om_k, w_k, Q = sp.symbols(
    'x_l y_l x_k y_k d_s x y x_k-1 y_k-1 theta_k theta_k-1 T v_k om_k w_k Q')
sp.init_printing(use_latex=True)

In [ ]:
x_process = sp.Matrix([[x_k1], [y_k1], [theta_k1]]) + T*sp.Matrix([[cos(theta_k1), 0],[sin(theta_k1),0],[0,1]])@(
    sp.Matrix([[v_k], [om_k]]))
#state = sp.Matrix([x,y,theta])
process_state = sp.Matrix([x_k1,y_k1,theta_k1])
x_process

In [ ]:
process_noise_state = sp.Matrix([v_k, om_k]) #using v and om since they have similar jac
L_k = x_process.jacobian(process_noise_state)
L_k

In [ ]:

F_k = x_process.jacobian(process_state)
F_k

In [ ]:
y_p = sp.Matrix([[sqrt((x_l - x_k - d_s*cos(theta_k))**2 + (y_l - y_k - d_s*sin(theta_k))**2)],
                 [(atan2(y_l - y_k - d_s*sin(theta_k), x_l - x_k - d_s*cos(theta_k))) - theta_k]])


In [ ]:
# Simplify by sub-ing d since d= 0
y_meas = y_p.subs(d_s,d)
y_meas

In [ ]:
var_meas = sp.Matrix([x_k,y_k,theta_k])
H_k = y_meas.jacobian(var_meas)
H_k

In [ ]:
M_k = np.eye(2)
M_k

In [ ]:
##LAMBDIFY APPROACH
# This approach uses the lambdify function in sympy to create functions
# Values can be passed to these functions

state_prev = np.array([[x_init,y_init,th_init]])
state_list = np.array([[50.0,0,1.5707963267948966]])

# Define f as a fxn of the variables f(x_k1,... ) = x_process
f = sp.lambdify((x_k1,y_k1,theta_k1,v_k,om_k,T), x_process,"numpy")

# Define Process jacobian in Symbols form
process_state = sp.Matrix([x_k1,y_k1,theta_k1])
process_noise_state = sp.Matrix([v_k, om_k]) #using v and om since they have similar jac

F_k_sp = x_process.jacobian(process_state)
print(F_k_sp)
L_k_sp = x_process.jacobian(process_noise_state)
print(F_k_sp)

# Define q as a fxn of the variables q(x_k1,... ) = H_k_sp for process Jacobian
f_f = sp.lambdify((theta_k1, v_k,T), F_k_sp, "numpy")

# Define f_l as a fxn of the variables f_l(x_k1,... ) = H_k_sp for process Jacobian
f_l = sp.lambdify((theta_k1,T), L_k_sp,"numpy")



t_prev = 0
dt = 0

for i in range(1,len(t)):
    dt = t[i]-t_prev   # elapsed time since last time step/stamp
    
    # Pass variables to fxn f(var,..)
    state_curr = f(x_init,y_init,th_init,v[i],om[i],dt).T
    
    # Append data to array for storing values
    state_list = np.append(state_list,state_curr,axis=0)
    
    # Update variables for next iteration
    x_init, y_init, th_init = state_curr[0,0], state_curr[0,1], state_curr[0,2]
    t_prev = t[i]
    
print(state_list)
plt.plot(state_list[:,0],state_list[:,1])
# plt.plot(t,state_list[:,2])

# reset values for easy debugging
x_init, y_init, th_init = 50.0, 0, 1.5707963267948966

### Initializing Parameters

Now that our data is loaded, we can begin getting things set up for our solver. One of the
most important aspects of designing a filter is determining the input and measurement noise covariance matrices, as well as the initial state and covariance values. We set the values here:

In [ ]:
v_var = 0.01  # translation velocity variance  
om_var = 0.01  # rotational velocity variance 
r_var = 0.001  # range measurements variance
b_var = 0.001  # bearing measurement variance

Q_km = np.diag([v_var, om_var]) # input noise covariance 
cov_y = np.diag([r_var, b_var])  # measurement noise covariance 

x_est = np.zeros([len(v), 3])  # estimated states, x, y, and theta
P_est = np.zeros([len(v), 3, 3])  # state covariance matrices

x_est[0] = np.array([x_init, y_init, th_init]) # initial state
P_est[0] = np.diag([1, 1, 0.1]) # initial state covariance

In [ ]:
# Wraps angle to (-pi,pi] range
# Initial wraptopi function was wrong and thus changed
def wraptopi(x):
    
    (((x / np.pi + 1) % 2 - 1) * np.pi)
    if x > np.pi:
        x = (((x / np.pi + 1) % 2 - 1) * np.pi)
#         x = x - (np.floor(x / (2 * np.pi)) + 1) * 2 * np.pi
    elif x < -np.pi:
        x = (((x / np.pi - 1) % 2 - 1) * np.pi)
#         x = x + (np.floor(x / (-2 * np.pi)) + 1) * 2 * np.pi
    return x

**Remember:** that it is neccessary to tune the measurement noise variances `r_var`, `b_var` in order for the filter to perform well!

In order for the orientation estimates to coincide with the bearing measurements, it is also neccessary to wrap all estimated $\theta$ values to the $(-\pi , \pi]$ range.

In [ ]:
# Define functions for process and measurement functions;
# and Jacobians

def calc_xk(x_prev,y_prev,th_prev,v_curr,om_curr,delta_T):
    xk = np.zeros([3,1])
    xk[0,0] = delta_T * v_curr*cos((th_prev)) + x_prev
    xk[1,0] = delta_T * v_curr*sin((th_prev)) + y_prev
    xk[2,0] = (delta_T * om_curr) + (th_prev)
    xk[2,0] =  wraptopi(xk[2,0])
    return xk
    
def calc_yk(x_lm,y_lm,x_curr,y_curr, th_curr):
    
    yk = np.zeros([2,1])
    yk[0,0] = sqrt((x_lm-x_curr)**2 +(y_lm-y_curr)**2)
    yk[1,0] = (np.arctan2( (y_lm-y_curr),(x_lm-x_curr))) - ( th_curr)
    yk[1,0] = wraptopi(yk[1,0])
    return yk
    
def calc_Fk(th_prev,v_curr,delta_T):
    fk = np.eye(3)
    fk[0,2] = -1 * delta_T * v_curr * sin((th_prev))
    fk[1,2] = 1 * delta_T * v_curr * cos((th_prev))
    return fk

def calc_Lk(th_prev,delta_T):
    lk = np.zeros([3,2])
    lk[0,0] = 1 * delta_T * cos((th_prev))
    lk[1,0] = 1 * delta_T * sin((th_prev))
    lk[2,1] = 1 * delta_T
    return lk

def calc_Hk(x_lm,y_lm,x_curr,y_curr):
    hk = np.zeros([2,3])
    hk[0,0] = (1*(x_curr-x_lm))/(sqrt((x_lm-x_curr)**2 +(y_lm-y_curr)**2))
    hk[0,1] = (1*(y_curr-y_lm))/(sqrt((x_lm-x_curr)**2 +(y_lm-y_curr)**2))
    hk[0,2] = 0
    hk[1,0] = (-1*(y_curr-y_lm))/((x_lm-x_curr)**2 +(y_lm-y_curr)**2)
    hk[1,1] = (-1*(-x_curr-x_lm))/((x_lm-x_curr)**2 +(y_lm-y_curr)**2)
    hk[1,2] = -1
    return hk

def calc_Mk():
    return np.eye(2)

In [ ]:
x_init, y_init, th_init = 50.0, 0, 1.5707963267948966
state_list = np.array([[50.0,0,1.5707963267948966]])
t_prev = 0
P_check = np.diag([1, 1, 0.1]) # initial state covariance
state_meas_list = np.array([[50.0,0,1.5707963267948966]])
print("================================STARTING========================")
for i in range(1,len(t)):
    

    dt = t[i]-t_prev   # elapsed time since last time step/stamp
    
    
    state_curr = calc_xk(x_init,y_init,th_init,v[i],om[i],dt).T
    
    
    F_km = calc_Fk(th_init,v[i],dt)
    
    L_km = calc_Lk(th_init,dt)
    
    P_check = (F_km@P_check@F_km.T) + (L_km@Q_km@L_km.T) # initial state covariance

    x_meas = state_curr
    
    for k in range(len(b[i])):
        
        x0, x1 = x_meas[0,0], x_meas[0,1]
        x2 = (x_meas[0,2])
        lk = l[k]
        l_0, l_1 = lk[0], lk[1]
        
        y_obs = np.array([[r[i,k]],
                          [b[i,k]]])
        
        y_calc = calc_yk(l_0,l_1,x0,x1, x2)
            
        y_diff = y_obs-y_calc
        y_diff[1,0] = wraptopi(y_diff[1,0])
        H_km = calc_Hk(l_0,l_1,x0,x1)
        M_k = calc_Mk()
        
        K_k = P_check@H_km.T@(inv((H_km@P_check@H_km.T + M_k@cov_y@M_k.T)))
        
        P_check = (np.eye(3)-(K_k@H_km))@(P_check)
        
        
        x_term = (K_k@(y_diff)).T
        x_meas = x_meas + x_term
        x_meas[0,2] = wraptopi(x_meas[0,2])
        
        
        
    
    state_curr = x_meas
    state_meas_list = np.append(state_meas_list,x_meas,axis=0)
    
    # Append data to array for storing values
    state_list = np.append(state_list,state_curr,axis=0)
    
    # Update variables for next iteration
    x_init, y_init, th_init = state_curr[0,0], state_curr[0,1], (state_curr[0,2])
    t_prev = t[i]

print("=======================COMPLETED========================")
# reset values for easy debugging
x_init, y_init, th_init = 50.0, 0, 1.5707963267948966

In [ ]:
e_fig = plt.figure()
ax = e_fig.add_subplot(111)
# ax.plot(state_list[:,0],state_list[:,1])
ax.plot(t[:],state_meas_list[:,2])
ax.set_xlabel('x [m]')
ax.set_ylabel('y [m]')
ax.set_title('Estimated trajectory')
plt.show()

e_fig = plt.figure()
ax = e_fig.add_subplot(111)
ax.plot(state_meas_list[:,0],state_meas_list[:,1])
ax.set_xlabel('x [m]')
ax.set_ylabel('y [m]')
ax.set_title('Estimated trajectory')
plt.show()


## Correction Step
-----
First, let's implement the measurement update function, which takes an available landmark measurement $l$ and updates the current state estimate $\mathbf{\check{x}}_k$.
For each landmark measurement received at a given timestep $k$, you should implement the following steps:

- Compute the measurement model Jacobians at $\mathbf{\check{x}}_{k}$
\begin{align}
\mathbf{y}^l_k = &\mathbf{h}(\mathbf{x}_{k}, \mathbf{n}^l_k) \\\\
\mathbf{H}_{k} = \frac{\partial \mathbf{h}}{\partial \mathbf{x}_{k}}\bigg|_{\mathbf{\check{x}}_{k},0}& \, , \, \, \, \,
\mathbf{M}_{k} = \frac{\partial \mathbf{h}}{\partial \mathbf{n}_{k}}\bigg|_{\mathbf{\check{x}}_{k},0} \, .
\end{align}
- Compute the Kalman Gain
\begin{align}
\mathbf{K}_k &= \mathbf{\check{P}}_k \mathbf{H}_k^T \left(\mathbf{H}_k \mathbf{\check{P}}_k \mathbf{H}_k^T + \mathbf{M}_k \mathbf{R}_k \mathbf{M}_k^T \right)^{-1} 
\end{align}
- Correct the predicted state
\begin{align}
\mathbf{\check{y}}^l_k &= \mathbf{h}\left(\mathbf{\check{x}}_k, \mathbf{0}\right) \\
\mathbf{\hat{x}}_k &= \mathbf{\check{x}}_k + \mathbf{K}_k \left(\mathbf{y}^l_k - \mathbf{\check{y}}^l_k\right)
\end{align}
- Correct the covariance
\begin{align}
\mathbf{\hat{P}}_k &= \left(\mathbf{I} - \mathbf{K}_k \mathbf{H}_k \right)\mathbf{\check{P}}_k
\end{align}

In [ ]:
def measurement_update(lk, rk, bk, P_check, x_check):
    
    # 1. Compute measurement Jacobian
    
    x0, x1, x2 = x_check[0,0], x_check[0,1], x_check[0,2]
    l_x, l_y = lk[0], lk[1]
    
    H_km = calc_Hk(l_x, l_y,x0,x1)
    M_k = calc_Mk()

    # 2. Compute Kalman Gain
    K_k = P_check@H_km.T@(inv((H_km@P_check@H_km.T + M_k@cov_y@M_k.T)))
    
    
    # 3. Correct predicted state (remember to wrap the angles to [-pi,pi])
    y_obs = np.array([[rk],[wraptopi(bk)]])
    y_calc = calc_yk(l_x, l_y, x0, x1, x2)
    
    y_diff = y_obs-y_calc
    y_diff[1,0] = wraptopi(y_diff[1,0])
    
    x_check = x_check + (K_k@(y_diff)).T
    x_check[0,2] = wraptopi(x_check[0,2])

    # 4. Correct covariance
    P_check = (np.eye(3)-(K_k@H_km))@(P_check)
    
    return x_check, P_check


## Prediction Step
-----
Now, implement the main filter loop, defining the prediction step of the EKF using the motion model provided:

\begin{align}
\mathbf{\check{x}}_k &= \mathbf{f}\left(\mathbf{\hat{x}}_{k-1}, \mathbf{u}_{k-1}, \mathbf{0} \right) \\
\mathbf{\check{P}}_k &= \mathbf{F}_{k-1}\mathbf{\hat{P}}_{k-1}\mathbf{F}_{k-1}^T + \mathbf{L}_{k-1}\mathbf{Q}_{k-1}\mathbf{L}_{k-1}^T \, .
\end{align}

Where

\begin{align}
\mathbf{F}_{k-1} = \frac{\partial \mathbf{f}}{\partial \mathbf{x}_{k-1}}\bigg|_{\mathbf{\hat{x}}_{k-1},\mathbf{u}_{k},0}  \, , \, \, \, \,
\mathbf{L}_{k-1} = \frac{\partial \mathbf{f}}{\partial \mathbf{w}_{k}}\bigg|_{\mathbf{\hat{x}}_{k-1},\mathbf{u}_{k},0} \, .
\end{align}

In [ ]:
# reset values for easy debugging
x_init, y_init, th_init =  50.0, 0, 1.5707963267948966
x_check = np.array([[x_init,y_init,th_init]])
P_check = np.diag([1, 1, 0.1]) # initial state covariance

#### 5. Main Filter Loop #######################################################################
print("======================= STARTING ========================")
for k in range(1, len(t)):  # start at 1 because we've set the initial prediciton

    delta_t = t[k] - t[k - 1]  # time step (difference between timestamps)
#     print("Iteration number: ",k)

    # 1. Update state with odometry readings (remember to wrap the angles to [-pi,pi])
    x_check = calc_xk(x_init,y_init,th_init,v[k],om[k],delta_t).T
    
    # 2. Motion model jacobian with respect to last state
    F_km = calc_Fk(th_init,v[k],delta_t)

    # 3. Motion model jacobian with respect to noise
    L_km = calc_Lk(th_init,delta_t)
    
    # 4. Propagate uncertainty
    P_check = (F_km@P_check@F_km.T) + (L_km@Q_km@L_km.T) # initial state covariance
    
    
    # 5. Update state estimate using available landmark measurements
    for i in range(len(r[k])):
        x_check, P_check = measurement_update(l[i], r[k, i], b[k, i], P_check, x_check)

    # Set final state predictions for timestep
    print(x_check)
    x_est[k, 0] = x_check[0,0]
    x_est[k, 1] = x_check[0,1]
    x_est[k, 2] = x_check[0,2]
    P_est[k, :, :] = P_check
    
    # Update initial states
    
    x_init, y_init, th_init = x_check[0,0], x_check[0,1], x_check[0,2]
    
print("======================= COMPLETED ========================")

Let's plot the resulting state estimates:

In [ ]:
e_fig = plt.figure()
ax = e_fig.add_subplot(111)
ax.plot(x_est[:, 0], x_est[:, 1])
ax.set_xlabel('x [m]')
ax.set_ylabel('y [m]')
ax.set_title('Estimated trajectory')
plt.show()

e_fig = plt.figure()
ax = e_fig.add_subplot(111)
ax.plot(t[:], x_est[:, 2])
ax.set_xlabel('Time [s]')
ax.set_ylabel('theta [rad]')
ax.set_title('Estimated trajectory')
plt.show()

Are you satisfied wth your results? The resulting trajectory should closely resemble the ground truth, with minor "jumps" in the orientation estimate due to angle wrapping. If this is the case, run the code below to produce your solution file.

In [31]:
with open('submission.pkl', 'wb') as f:
    pickle.dump(x_est, f, pickle.HIGHEST_PROTOCOL)